In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from IPython.display import display, Markdown

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql import SQLContext

## LOADING DATAFRAMES 

To answer the business questions that were lined out in our presentation we focus on the most important tables: _Customers, Items, Payments, Reviews, Orders, Products and Sellers_. In the following they are all loaded.

In [5]:
customers_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_customers_dataset.csv")

In [6]:
customers_df.show(truncate=True)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [7]:
order_items_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_order_items_dataset.csv")

In [8]:
order_items_df.show(truncate=True)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [9]:
order_payments_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_order_payments_dataset.csv")

In [10]:
order_payments_df.show(5,truncate=True)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [11]:
order_reviews_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_order_reviews_dataset.csv")

In [12]:
order_reviews_df.show(5, truncate=True)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [13]:
display(order_reviews_df.take(1))

[Row(review_id='7bc2406110b926393aa56f80a40eba40', order_id='73fc7af87114b39712e6da79b0a377eb', review_score='4', review_comment_title=None, review_comment_message=None, review_creation_date='2018-01-18 00:00:00', review_answer_timestamp='2018-01-18 21:46:59')]

In [14]:
orders_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_orders_dataset.csv")

In [15]:
orders_df.show(5,truncate=True)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [16]:
products_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_products_dataset.csv")

In [17]:
products_df.show(5, truncate=True)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [18]:
sellers_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("olist_sellers_dataset.csv")

In [19]:
sellers_df.show(truncate=True)

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|
|ccc4bbb5f32a6ab2b...|                 80310|         curitiba|          PR|

In [20]:
product_cat_translation_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .csv("product_category_name_translation.csv")

## TRANSLATION PRODUCT CATEGORY FROM PORTUGESE TO ENGLISH

In [21]:
product_cat_translation_df.show(truncate=True)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
|    alimentos_bebidas|                   food_drink|
|                bebes|                         baby|
|            papelaria|                   stationery|
| tablets_impressao...|         tablets_printing_...|
|           brinquedos|                         toys|
|       telefonia_fixa|     

In [22]:
products = products_df.join(product_cat_translation_df,"product_category_name")\
                    .select("product_id", "product_category_name_english","product_description_lenght", "product_photos_qty")\
                    .withColumnRenamed("product_category_name_english", "product_name")
products.show(10)

+--------------------+-------------------+--------------------------+------------------+
|          product_id|       product_name|product_description_lenght|product_photos_qty|
+--------------------+-------------------+--------------------------+------------------+
|1e9e8ef04dbcff454...|          perfumery|                       287|                 1|
|3aa071139cb16b67c...|                art|                       276|                 1|
|96bd76ec8810374ed...|     sports_leisure|                       250|                 1|
|cef67bcfe19066a93...|               baby|                       261|                 1|
|9dc1a7de274444849...|         housewares|                       402|                 4|
|41d3672d4792049fa...|musical_instruments|                       745|                 1|
|732bd381ad09e530f...|         cool_stuff|                      1272|                 4|
|2548af3e6e77a690c...|    furniture_decor|                       184|                 2|
|37cc742be07708b53...

## REVIEW CATEGORIES PER ORDER

In [23]:
order_info = order_payments_df.join(order_reviews_df,"order_id")\
                            .select("order_id","payment_type","payment_installments", "payment_value", "review_score")\
                            .where(col("review_score")>0)\
                            .where(col("review_score")<6)\
                            .withColumn("review", when(col("review_score")==5, "Excellent")
                                                .when(col("review_score")==4, "Very Good")
                                                .when(col("review_score")==3, "Good")
                                                .when(col("review_score")==2, "Unsatisfied")
                                                .when(col("review_score")==1, "Very Unsatisfied")
                                                .when(col("review_score")==0, "Disappointing"))

order_info.show(10)

+--------------------+------------+--------------------+-------------+------------+----------------+
|            order_id|payment_type|payment_installments|payment_value|review_score|          review|
+--------------------+------------+--------------------+-------------+------------+----------------+
|b81ef226f3fe1789b...| credit_card|                   8|        99.33|           1|Very Unsatisfied|
|a9810da82917af2d9...| credit_card|                   1|        24.39|           5|       Excellent|
|25e8ea4e93396b6fa...| credit_card|                   1|        65.71|           5|       Excellent|
|ba78997921bbcdc13...| credit_card|                   8|       107.78|           5|       Excellent|
|ba78997921bbcdc13...| credit_card|                   8|       107.78|           5|       Excellent|
|42fdf880ba16b47b5...| credit_card|                   2|       128.45|           5|       Excellent|
|298fcdf1f73eb413e...| credit_card|                   2|        96.12|           5|       E

In [24]:
payment_type_review=order_info.where(col("payment_type")!="not_defined")\
        .groupBy("payment_type")\
        .agg(avg("payment_installments").alias("Avg_paymnt_install"),\
            sum("payment_value").alias("Total_Payment"),
            avg("review_score").alias("Avg_review"),
            max("review_score").alias("Best_review"),
            min("review_score").alias("Worst_review"),
            max("payment_installments").alias("Max_Install"),
            min("payment_installments").alias("Min_Install"))\
        .orderBy(col("Total_Payment").desc())\
        .show(10, truncate=True)

+------------+------------------+------------------+-----------------+-----------+------------+-----------+-----------+
|payment_type|Avg_paymnt_install|     Total_Payment|       Avg_review|Best_review|Worst_review|Max_Install|Min_Install|
+------------+------------------+------------------+-----------------+-----------+------------+-----------+-----------+
| credit_card| 3.509477806788512|1.24837152499995E7|4.087637075718016|          5|           1|         24|          0|
|      boleto|               1.0| 2854996.229999999|4.086630907802854|          5|           1|          1|          1|
|     voucher|               1.0|374780.95000000065|4.003804253847484|          5|           1|          1|          1|
|  debit_card|               1.0|218161.98000000019|4.168083714846305|          5|           1|          1|          1|
+------------+------------------+------------------+-----------------+-----------+------------+-----------+-----------+



The average number of installments and sum of total payment grouped by review categories. 

In [25]:
review_info=order_info.groupBy("review")\
        .agg(avg("payment_installments").alias("Avg_paymnt_install"),\
            sum("payment_value").alias("Total_Payment"))\
        .orderBy(col("Total_Payment").desc())

for c in ["Avg_paymnt_install", "Total_Payment"]:
    review_info = review_info.withColumn(c, round(c, 2))

review_info.show(10)

+----------------+------------------+-------------+
|          review|Avg_paymnt_install|Total_Payment|
+----------------+------------------+-------------+
|       Excellent|              2.83|   8955199.96|
|       Very Good|              2.75|   2960722.31|
|Very Unsatisfied|              3.09|   2231811.62|
|            Good|              2.87|   1239884.94|
|     Unsatisfied|              2.96|    544035.58|
+----------------+------------------+-------------+



## ANALYZING ORDERS AND SELLERS

In [26]:
order_items_sellers = order_items_df.join(sellers_df, "seller_id")\
                                    .select("order_id", "order_item_id", "product_id", "seller_id", "seller_city", "seller_state","price","freight_value")\

order_items_sellers.show(5, truncate=True)

+--------------------+-------------+--------------------+--------------------+-------------+------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|  seller_city|seller_state|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------+------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|volta redonda|          SP| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|    sao paulo|          SP|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|borda da mata|          MG|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|       franca|          SP|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|       loanda|          PR|199.9|       

In [27]:
order_items_sellers.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [28]:
summary_sellers = order_items_sellers.select("price", "freight_value").summary()

for c in ["price", "freight_value"]:
    summary_sellers = summary_sellers.withColumn(c, round(c, 2))

summary_sellers.show(10)

+-------+--------+-------------+
|summary|   price|freight_value|
+-------+--------+-------------+
|  count|112650.0|     112650.0|
|   mean|  120.65|        19.99|
| stddev|  183.63|        15.81|
|    min|    0.85|          0.0|
|    25%|    39.9|        13.08|
|    50%|   74.99|        16.26|
|    75%|   134.9|        21.15|
|    max|  6735.0|       409.68|
+-------+--------+-------------+



## TOTAL ORDERS, DELIVERY COSTS AND AVERAGE_COSTS PER SELLER

In the following, a table is created that provides more information on the sellers. It shows basic metrics such as the Total Price, Tota Delivery Costs, Average Delivery Costs, and Total Order for each seller.

In [29]:
info_by_seller = order_items_sellers.groupBy("seller_id")\
                    .agg(sum("price").alias("total_price"),\
                        sum("freight_value").alias("Total_Delivery_Costs"),
                        avg("freight_value").alias("Delivery_Costs_Avg"),
                        countDistinct("order_id").alias("No_Orders_Sold"))\
                    .orderBy(col("No_Orders_Sold").desc())

for c in ['total_price','Total_Delivery_Costs','Delivery_Costs_Avg','No_Orders_Sold']:
    info_by_seller = info_by_seller.withColumn(c, round(c, 2))

info_by_seller.show(10)

+--------------------+-----------+--------------------+------------------+--------------+
|           seller_id|total_price|Total_Delivery_Costs|Delivery_Costs_Avg|No_Orders_Sold|
+--------------------+-----------+--------------------+------------------+--------------+
|6560211a19b47992c...|  123304.83|            27960.94|             13.75|          1854|
|4a3ca9315b744ce9f...|  200472.92|            35067.04|             17.65|          1806|
|cc419e0650a3c5ba7...|  104288.42|            25668.99|             14.46|          1706|
|1f50f920176fa81da...|  106939.21|            35165.77|             18.21|          1404|
|da8622b14eb17ae28...|  160236.57|            24955.75|             16.09|          1314|
|955fee9216a65b617...|   135171.7|            25430.98|             16.97|          1287|
|7a67c85e85bb2ce85...|  141745.53|            20902.85|             17.85|          1160|
|ea8482cd71df3c196...|   37177.52|            17544.77|             14.58|          1146|
|4869f7a5d

## TOTAL ORDERS, DELIVERY COSTS AND AVERAGE_COSTS PER CITY

In [30]:
info_by_city = order_items_sellers.groupBy("seller_city")\
                    .agg(sum("price").alias("Total_price"),\
                        sum("freight_value").alias("Total_Delivery_Costs"),
                        avg("freight_value").alias("Delivery_Costs_Avg"),
                        countDistinct("order_id").alias("No_Orders_Selled"))\
                    .orderBy(col("No_Orders_Selled").desc())


for c in ['Total_price','Total_Delivery_Costs','Delivery_Costs_Avg','No_Orders_Selled']:
    info_by_city = info_by_city.withColumn(c, round(c, 2))

info_by_city.show(10)

+--------------------+-----------+--------------------+------------------+----------------+
|         seller_city|Total_price|Total_Delivery_Costs|Delivery_Costs_Avg|No_Orders_Selled|
+--------------------+-----------+--------------------+------------------+----------------+
|           sao paulo| 2702878.14|           478867.99|             17.11|           24588|
|            ibitinga|  624592.94|           137681.07|             17.77|            6565|
|            curitiba|  470759.82|            71116.02|             23.58|            2722|
|         santo andre|   228561.6|            46249.18|              15.6|            2711|
|      belo horizonte|  207671.23|            46556.88|             17.95|            2414|
|      rio de janeiro|  358413.59|            46224.29|             18.93|            2193|
|           guarulhos|  329494.38|            47287.87|             20.02|            2071|
|      ribeirao preto|  275976.44|            38705.73|             17.06|      

## TOTAL ORDERS, DELIVERY COSTS AND AVERAGE_COSTS PER STATE

In [31]:
info_by_state = order_items_sellers.groupBy("seller_state")\
                    .agg(sum("price").alias("Total_price"),\
                        sum("freight_value").alias("Total_Delivery_Costs"),
                        avg("freight_value").alias("Delivery_Costs_Avg"),
                        count("order_id").alias("No_Orders_Selled"))\
                    .orderBy(col("No_Orders_Selled").desc())

for c in ['Total_price','Total_Delivery_Costs','Delivery_Costs_Avg','No_Orders_Selled']:
    info_by_state = info_by_state.withColumn(c, round(c, 2))

info_by_state.show(10)

+------------+-----------+--------------------+------------------+----------------+
|seller_state|Total_price|Total_Delivery_Costs|Delivery_Costs_Avg|No_Orders_Selled|
+------------+-----------+--------------------+------------------+----------------+
|          SP| 8753396.21|          1482487.67|             18.45|           80342|
|          MG| 1011564.74|           212595.06|             24.08|            8827|
|          PR| 1261887.21|           197013.52|             22.72|            8671|
|          RJ|  843984.22|             93829.9|             19.47|            4818|
|          SC|  632426.07|           106547.06|             26.15|            4075|
|          RS|  378559.54|            57243.09|             26.03|            2199|
|          DF|   97749.48|            18494.06|             20.57|             899|
|          BA|  285561.56|            19700.68|             30.64|             643|
|          GO|   66399.21|             12565.5|             24.16|          

# Which are our top 10 sellers based on the rating?

In [32]:
sellers_review = order_items_sellers.join(order_info, "order_id")\
                                    .select("seller_id", "seller_city", "seller_state", "payment_value", 
                                            "price","freight_value", "review_score", "review")

sellers_review.show(10)

+--------------------+---------------+------------+-------------+------+-------------+------------+---------+
|           seller_id|    seller_city|seller_state|payment_value| price|freight_value|review_score|   review|
+--------------------+---------------+------------+-------------+------+-------------+------------+---------+
|325f3178fb58e2a97...|taboao da serra|          SP|        78.43|  27.9|         3.81|           5|Excellent|
|a17f621c590ea0fab...|       sorocaba|          SP|        78.43| 21.33|        25.39|           5|Excellent|
|1b4c3a6f53068f0b6...|    sao ludgero|          SC|        188.4| 159.9|         28.5|           5|Excellent|
|ea8482cd71df3c196...|      sao paulo|          SP|        50.09| 34.99|         15.1|           3|     Good|
|d1c281d3ae1492323...|       ibitinga|          SP|       169.76|151.99|        17.77|           5|Excellent|
|c8b3445d737de6bef...|      sao paulo|          SP|        61.72| 49.99|        11.73|           5|Excellent|
|6560211a1

## TOP SELLERS ACCORDING TO THEIR REVENUE AND THE AVG REVIEW

In [33]:
top_sellers_review = sellers_review.groupBy("seller_id")\
                                    .agg(sum("price").alias("Total_Revenue"), 
                                        sum("freight_value").alias("Delivery_Costs"), 
                                        avg("review_score").alias("avg_review"))\
                                    .withColumn("Total_Product_Cost", col("Total_Revenue")+col("Delivery_Costs"))\
                                    .orderBy(col("Total_Revenue").desc())

for c in ['Total_Revenue','Delivery_Costs','avg_review', 'Total_Product_Cost']:
    top_sellers_review = top_sellers_review.withColumn(c, round(c, 2))

top_sellers_review.show(10)

+--------------------+-------------+--------------+----------+------------------+
|           seller_id|Total_Revenue|Delivery_Costs|avg_review|Total_Product_Cost|
+--------------------+-------------+--------------+----------+------------------+
|53243585a1d6dc264...|    242591.55|       14137.8|      4.07|         256729.35|
|4869f7a5dfa277a7d...|    236465.64|      20621.79|      4.11|         257087.43|
|4a3ca9315b744ce9f...|    213208.27|      37258.93|       3.8|          250467.2|
|fa1c13f2614d7b5c4...|    202716.62|      10398.21|      4.34|         213114.83|
|7c67e1448b00f6e96...|     198118.3|      54262.15|      3.39|         252380.45|
|7e93a43ef30c4f03f...|    182648.18|        6560.8|      4.21|         189208.98|
|da8622b14eb17ae28...|    171055.17|      26704.13|      4.07|          197759.3|
|7a67c85e85bb2ce85...|    149554.88|      21981.23|      4.23|         171536.11|
|1025f0e2d44d7041d...|    142646.77|      34792.25|      3.86|         177439.02|
|955fee9216a65b6

## TOP SELLERS ACCORDING TO THE REVENUE COMPARED WITH THE REVIEW

In [34]:
top_sellers_review_2 = sellers_review.groupBy("seller_id")\
                                    .pivot("review")\
                                    .agg(sum("price"))\
                                    .withColumn("Total_Revenue", col("Excellent")+col("Good")+col("Unsatisfied")+col("Very Good")+col("Very Unsatisfied"))\
                                    .orderBy(col("Total_Revenue").desc())

for c in ['Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied", "Total_Revenue"]:
    top_sellers_review_2 = top_sellers_review_2.withColumn(c, round(c, 2))

top_sellers_review_2.show(10)

+--------------------+---------+--------+-----------+---------+----------------+-------------+
|           seller_id|Excellent|    Good|Unsatisfied|Very Good|Very Unsatisfied|Total_Revenue|
+--------------------+---------+--------+-----------+---------+----------------+-------------+
|53243585a1d6dc264...|141474.66|17910.06|     2898.8| 57082.38|        23225.65|    242591.55|
|4869f7a5dfa277a7d...|139216.53| 19755.0|    5513.97| 44011.74|         27968.4|    236465.64|
|4a3ca9315b744ce9f...|101326.21|28196.73|   11138.55|  40481.2|        32065.58|    213208.27|
|fa1c13f2614d7b5c4...|142597.21| 7702.14|    4793.25| 32375.62|         15248.4|    202716.62|
|7c67e1448b00f6e96...| 64983.95|29442.83|   11836.68| 47464.44|         44390.4|     198118.3|
|7e93a43ef30c4f03f...|116963.43|12803.81|    1386.35| 30074.67|        21419.92|    182648.18|
|da8622b14eb17ae28...| 95365.43| 14500.4|     7778.4| 32629.07|        20781.87|    171055.17|
|7a67c85e85bb2ce85...| 92312.91|12139.98|    4813.

### BEST SELLERS ACCORDING TO THE NUMBER OF REVIEWS

In [35]:
top_sellers_review_3 = sellers_review.groupBy("seller_id")\
                                    .pivot("review")\
                                    .agg(count("review_score").alias("No_Reviews"))\
                                    .withColumn("Total_Reviews", col("Excellent")+col("Very Good")+col("Good")+
                                                col("Unsatisfied")+col("Very Unsatisfied"))\
                                    .orderBy(col("Excellent").desc(), col("Very Good").desc(), col("Good").desc())

for c in ['Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied"]:
    top_sellers_review_3 = top_sellers_review_3.withColumn(c, round(c, 2))

top_sellers_review_3.show(10)

+--------------------+---------+----+-----------+---------+----------------+-------------+
|           seller_id|Excellent|Good|Unsatisfied|Very Good|Very Unsatisfied|Total_Reviews|
+--------------------+---------+----+-----------+---------+----------------+-------------+
|1f50f920176fa81da...|     1134| 151|         69|      361|             294|         2009|
|cc419e0650a3c5ba7...|     1091| 178|         59|      324|             233|         1885|
|6560211a19b47992c...|     1069| 230|        101|      435|             276|         2111|
|4a3ca9315b744ce9f...|     1021| 274|        103|      409|             321|         2128|
|da8622b14eb17ae28...|      948| 146|         66|      312|             184|         1656|
|955fee9216a65b617...|      817| 149|         63|      335|             153|         1517|
|7a67c85e85bb2ce85...|      752| 102|         38|      246|              98|         1236|
|1025f0e2d44d7041d...|      750| 121|         63|      296|             235|         1465|

### BEST SELLERS ACCORDING TO THE NUMBER OF REVIEWS AND THEIR RATINGS

In [36]:
best_sellers = top_sellers_review_3.withColumn("Excellent_Rate", col("Excellent")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Good_Rate", col("Very Good")/col("Total_Reviews")*100)\
                                    .withColumn("Good_Rate", col("Good")/col("Total_Reviews")*100)\
                                    .withColumn("Unsatisfied_Rate", col("Unsatisfied")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Unsatisfied_Rate", col("Very Unsatisfied")/col("Total_Reviews")*100)\
                                    .drop('Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied")\
                                    .orderBy(col("Excellent_Rate").desc())

for c in ['Excellent_Rate','Very_Good_Rate','Good_Rate','Unsatisfied_Rate', "Very_Unsatisfied_Rate"]:
    best_sellers = best_sellers.withColumn(c, round(c, 2))

best_sellers.show(10)

+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|           seller_id|Total_Reviews|Excellent_Rate|Very_Good_Rate|Good_Rate|Unsatisfied_Rate|Very_Unsatisfied_Rate|
+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|f7496d659ca9fdaf3...|           29|         82.76|           6.9|     3.45|            3.45|                 3.45|
|d9a84e1403de8da0c...|           67|          80.6|         10.45|     2.99|            1.49|                 4.48|
|b410bdd36d5db7a65...|           45|          80.0|         13.33|     2.22|            2.22|                 2.22|
|d921b68bf747894be...|           82|         78.05|         10.98|      6.1|            1.22|                 3.66|
|9d5a9018aee56acb3...|           86|         77.91|           9.3|     1.16|            3.49|                 8.14|
|0432ead42b6c8a0bd...|           95|         76.84|         12.63|     2

### WORST SELLERS - REVIEWS RATING

In [37]:
worst_sellers = top_sellers_review_3.withColumn("Excellent_Rate", col("Excellent")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Good_Rate", col("Very Good")/col("Total_Reviews")*100)\
                                    .withColumn("Good_Rate", col("Good")/col("Total_Reviews")*100)\
                                    .withColumn("Unsatisfied_Rate", col("Unsatisfied")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Unsatisfied_Rate", col("Very Unsatisfied")/col("Total_Reviews")*100)\
                                    .drop('Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied")\
                                    .orderBy(col("Very_Unsatisfied_Rate").desc())

for c in ['Excellent_Rate','Very_Good_Rate','Good_Rate','Unsatisfied_Rate', "Very_Unsatisfied_Rate"]:
    worst_sellers = worst_sellers.withColumn(c, round(c, 2))

worst_sellers.show(10)

+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|           seller_id|Total_Reviews|Excellent_Rate|Very_Good_Rate|Good_Rate|Unsatisfied_Rate|Very_Unsatisfied_Rate|
+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|1ca7077d890b907f8...|          137|         18.25|           7.3|     8.76|            6.57|                59.12|
|2709af9587499e95e...|           47|         31.91|          8.51|     2.13|            4.26|                53.19|
|10ff823eda2b032e2...|           17|         29.41|          5.88|     5.88|            5.88|                52.94|
|b5abf4f36adc04311...|           13|         23.08|         15.38|     7.69|            7.69|                46.15|
|b19f3ca2ea4759137...|           24|         33.33|          8.33|     8.33|            4.17|                45.83|
|26e2c91ef821e1ff8...|           18|         22.22|          5.56|    22

### TOP SELLERS ACCORDING TO THE NUMBER OF REVIEWS AND THEIR RATINGS

In [38]:
most_reviews = top_sellers_review_3.withColumn("Excellent_Rate", col("Excellent")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Good_Rate", col("Very Good")/col("Total_Reviews")*100)\
                                    .withColumn("Good_Rate", col("Good")/col("Total_Reviews")*100)\
                                    .withColumn("Unsatisfied_Rate", col("Unsatisfied")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Unsatisfied_Rate", col("Very Unsatisfied")/col("Total_Reviews")*100)\
                                    .drop('Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied")\
                                    .orderBy(col("Total_Reviews").desc())

for c in ['Excellent_Rate','Very_Good_Rate','Good_Rate','Unsatisfied_Rate', "Very_Unsatisfied_Rate"]:
    most_reviews = most_reviews.withColumn(c, round(c, 2))

most_reviews.show(10)

+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|           seller_id|Total_Reviews|Excellent_Rate|Very_Good_Rate|Good_Rate|Unsatisfied_Rate|Very_Unsatisfied_Rate|
+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|4a3ca9315b744ce9f...|         2128|         47.98|         19.22|    12.88|            4.84|                15.08|
|6560211a19b47992c...|         2111|         50.64|         20.61|     10.9|            4.78|                13.07|
|1f50f920176fa81da...|         2009|         56.45|         17.97|     7.52|            3.43|                14.63|
|cc419e0650a3c5ba7...|         1885|         57.88|         17.19|     9.44|            3.13|                12.36|
|da8622b14eb17ae28...|         1656|         57.25|         18.84|     8.82|            3.99|                11.11|
|955fee9216a65b617...|         1517|         53.86|         22.08|     9

In [44]:
most_reviews2 = top_sellers_review_3.withColumn("Excellent_Rate", col("Excellent")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Good_Rate", col("Very Good")/col("Total_Reviews")*100)\
                                    .withColumn("Good_Rate", col("Good")/col("Total_Reviews")*100)\
                                    .withColumn("Unsatisfied_Rate", col("Unsatisfied")/col("Total_Reviews")*100)\
                                    .withColumn("Very_Unsatisfied_Rate", col("Very Unsatisfied")/col("Total_Reviews")*100)\
                                    .drop('Excellent','Very Good','Good','Unsatisfied', "Very Unsatisfied")\
                                    .where(col("Total_Reviews")>100)\
                                    .orderBy(col("Excellent_Rate").desc())

for c in ['Excellent_Rate','Very_Good_Rate','Good_Rate','Unsatisfied_Rate', "Very_Unsatisfied_Rate"]:
    most_reviews2 = most_reviews2.withColumn(c, round(c, 2))

most_reviews2.show(10)

+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|           seller_id|Total_Reviews|Excellent_Rate|Very_Good_Rate|Good_Rate|Unsatisfied_Rate|Very_Unsatisfied_Rate|
+--------------------+-------------+--------------+--------------+---------+----------------+---------------------+
|ac3508719a1d8f5b7...|          105|         76.19|         14.29|     4.76|            0.95|                 3.81|
|289cdb325fb7e7f89...|          126|          75.4|         15.87|     3.97|            0.79|                 3.97|
|c3cfdc648177fdbbb...|          324|         74.38|         12.35|     5.25|            1.23|                 6.79|
|70c27847eca8195c9...|          103|         72.82|          7.77|     1.94|            5.83|                11.65|
|fa40cc5b934574b62...|          352|         72.73|         11.36|      7.1|            2.56|                 6.25|
|5cf13accae3222c70...|          163|         72.39|         15.34|     4

## Why these sellers are the best?

In [46]:
orders_sellers_products = order_items_sellers.join(products, "product_id")\
                                        .withColumnRenamed("product_description_lenght", "descrp_len")\
                                        .withColumnRenamed("product_photos_qty", "pics_qty")\
                                        .select("seller_id", "seller_city", "seller_state","price", 
                                                "freight_value", "product_name", "descrp_len","pics_qty")\
                                        .withColumn("Description", when(col("descrp_len")<150,"Brief")
                                                .when((col("descrp_len")>150) & (col("descrp_len")<500), "Short")
                                                .when((col("descrp_len")>500) & (col("descrp_len")<500), "Long")
                                                .when(col("descrp_len")>1000, "Detailed")
                                                .otherwise("Empty"))
orders_sellers_products.show(10)

+--------------------+--------------------+------------+------+-------------+--------------------+----------+--------+-----------+
|           seller_id|         seller_city|seller_state| price|freight_value|        product_name|descrp_len|pics_qty|Description|
+--------------------+--------------------+------------+------+-------------+--------------------+----------+--------+-----------+
|1660ad786c27fd935...|sao bernardo do c...|          SP| 130.0|        38.46|computers_accesso...|       978|       3|      Empty|
|620c87c171fb2a6dd...|          petropolis|          RJ|  79.9|        25.05|           perfumery|       259|       1|      Short|
|620c87c171fb2a6dd...|          petropolis|          RJ|  79.9|        14.73|           perfumery|       259|       1|      Short|
|620c87c171fb2a6dd...|          petropolis|          RJ|  79.9|        11.17|           perfumery|       259|       1|      Short|
|620c87c171fb2a6dd...|          petropolis|          RJ|  69.9|        24.98|      

## TOP SELLERS ACCORDING TO THE PRODUCTS SOLD

In [47]:
Sellers_by_total_products = orders_sellers_products.groupBy("seller_id")\
                    .agg(sum("price").alias("Total_Revenue"),
                        sum("freight_value").alias("Delivery_Cost"), 
                        avg("descrp_len").alias("avg_len_dscrp"), 
                        avg("pics_qty").alias("avg_pics"),
                        count("product_name").alias("No_Products"))\
                    .orderBy(col("Total_Revenue").desc())

for c in ['Total_Revenue','Delivery_Cost','avg_len_dscrp','avg_pics','No_Products']:
    Sellers_by_total_products = Sellers_by_total_products.withColumn(c, round(c, 2))

Sellers_by_total_products.show(10)

+--------------------+-------------+-------------+-------------+--------+-----------+
|           seller_id|Total_Revenue|Delivery_Cost|avg_len_dscrp|avg_pics|No_Products|
+--------------------+-------------+-------------+-------------+--------+-----------+
|4869f7a5dfa277a7d...|    229472.63|     20168.07|       523.83|    1.59|       1156|
|53243585a1d6dc264...|    222776.05|     13080.63|      2027.85|    3.06|        410|
|4a3ca9315b744ce9f...|    200472.92|     35067.04|       329.23|    1.13|       1987|
|fa1c13f2614d7b5c4...|    194042.03|      10042.7|       608.07|    1.85|        586|
|7c67e1448b00f6e96...|    187923.89|     51612.55|       1096.1|    1.06|       1364|
|7e93a43ef30c4f03f...|    176431.87|      6322.18|       501.27|    1.76|        340|
|da8622b14eb17ae28...|    160236.57|     24955.75|       244.22|    1.01|       1551|
|7a67c85e85bb2ce85...|    141745.53|     20902.85|       864.21|    1.26|       1171|
|1025f0e2d44d7041d...|    138968.55|     33892.14|    

## TOP SELLERS ACCORDING TO THE LENGTH DESCRIPTION

In [161]:
top_sellers = orders_sellers_products.groupBy("seller_id")\
                    .agg(sum("price").alias("Total_Revenue"),
                        sum("freight_value").alias("Delivery_Cost"), 
                        avg("descrp_len").alias("avg_len_dscrp"), 
                        avg("pics_qty").alias("avg_pics"),
                        count("product_name").alias("No_Products"))\
                    .orderBy(col("avg_len_dscrp").desc())

for c in ['Total_Revenue','Delivery_Cost','avg_len_dscrp','avg_pics','No_Products']:
    top_sellers = top_sellers.withColumn(c, round(c, 2))

top_sellers.show(10)

+--------------------+-------------+-------------+-------------+--------+-----------+
|           seller_id|Total_Revenue|Delivery_Cost|avg_len_dscrp|avg_pics|No_Products|
+--------------------+-------------+-------------+-------------+--------+-----------+
|491a5ada3aa167841...|       2840.0|        33.81|       3992.0|     1.0|          2|
|ee27a8f15b1dded4d...|       6499.0|       227.66|       3954.0|     2.0|          1|
|3ea00e1dea2b0659d...|        505.0|       144.97|       3908.0|   11.71|          7|
|1d29dfba02015238d...|        612.8|       177.12|       3892.0|     1.0|         12|
|2039c5e51785b5dde...|        159.9|        13.78|       3860.0|     1.0|          1|
|78ef7e885d17f35e2...|        388.0|        65.44|      3818.67|     1.0|          3|
|ace601496e211a664...|        165.0|        14.92|       3799.0|     5.0|          1|
|6235903333ba7de19...|        94.06|        41.42|       3748.0|     1.0|          1|
|b0f2a6122a90a93eb...|        456.6|        69.54|    

## HOW IMPORTANT IS THE DESCRIPTION AND THE PICTURES ATTACHED DUE TO THE TOTAL PRODUCTS SOLD

In [162]:
Description = orders_sellers_products.groupBy("Description")\
                    .agg(sum("price").alias("Total_Revenue"),
                        sum("freight_value").alias("Delivery_Cost"), 
                        avg("pics_qty").alias("avg_pics"),
                        count("product_name").alias("No_Products"))\
                    .orderBy(col("Total_Revenue").desc())

for c in ['Total_Revenue','Delivery_Cost','avg_pics','No_Products']:
    Description = Description.withColumn(c, round(c, 2))

Description.show(10)

+-----------+-------------+-------------+--------+-----------+
|Description|Total_Revenue|Delivery_Cost|avg_pics|No_Products|
+-----------+-------------+-------------+--------+-----------+
|   Detailed|   4688985.08|    620743.04|    2.45|      27108|
|      Empty|   4584046.32|    773349.83|     2.3|      39176|
|      Short|   3795274.47|    747288.25|    1.98|      40033|
|      Brief|    338288.07|     81915.28|     2.0|       4706|
+-----------+-------------+-------------+--------+-----------+



# CUSTOMERS ANALYSIS

For a first analysis, we want to double check how many orders were delivered correclty and how many they had a different end.

In [48]:
customer_info = orders_df.join(customers_df,"customer_id")\
                        .select("order_id", "order_status", "customer_zip_code_prefix", "customer_city", "customer_state")\
                        .withColumnRenamed("customer_zip_code_prefix", "customer_zipcode")
customer_info.show(10)                 

+--------------------+------------+----------------+--------------------+--------------+
|            order_id|order_status|customer_zipcode|       customer_city|customer_state|
+--------------------+------------+----------------+--------------------+--------------+
|e481f51cbdc54678b...|   delivered|            3149|           sao paulo|            SP|
|53cdb2fc8bc7dce0b...|   delivered|           47813|           barreiras|            BA|
|47770eb9100c2d0c4...|   delivered|           75265|          vianopolis|            GO|
|949d5b44dbf5de918...|   delivered|           59296|sao goncalo do am...|            RN|
|ad21c59c0840e6cb8...|   delivered|            9195|         santo andre|            SP|
|a4591c265e18cb1dc...|   delivered|           86320|        congonhinhas|            PR|
|136cce7faa42fdb2c...|    invoiced|           98900|          santa rosa|            RS|
|6514b8ad8028c9f2c...|   delivered|           26525|           nilopolis|            RJ|
|76c6e866289321a7c...

In [90]:
orders_df.groupBy("order_status")\
        .agg(count("customer_id").alias("No_orders"))\
        .orderBy(col("No_orders").desc())\
        .show(10)

+------------+---------+
|order_status|No_orders|
+------------+---------+
|   delivered|    96478|
|     shipped|     1107|
|    canceled|      625|
| unavailable|      609|
|    invoiced|      314|
|  processing|      301|
|     created|        5|
|    approved|        2|
+------------+---------+



Close to 97% of the orders were delivered correctly so the delivery service it's really good. Let's check if we can improve any step inside this delivery service. 

## RELIABILITY E-COMERCE PLATFORM

What we want to double-check it's the reliability of the e-commerce platform estimation dates delivery, and compare it with the real delivery date confirmed. 

For this, we need to de-structured the date columns to extract the info from the year, month, and day from the estimation date delivery, the purchased date, and finally the order delivered date. 

Once we have done this, we can get the difference between the purchased date and the delivery date to know how many days it took to be delivered to the customer. The same for the estimation date. 

The difference between these 2 columns was calculated for further categorization.

In [103]:
timeline_dates = orders_df.join(customers_df, "customer_id")\
                        .select("customer_id", "order_status", "order_delivered_customer_date", 
                                "order_purchase_timestamp", "order_estimated_delivery_date")\
                        .withColumn("Year_delivered", when(col("order_delivered_customer_date")!="null",year("order_delivered_customer_date")))\
                        .withColumn("Month_delivered", when(col("order_delivered_customer_date")!="null",month("order_delivered_customer_date")))\
                        .withColumn("Day_delivered", when(col("order_delivered_customer_date")!="null",dayofmonth("order_delivered_customer_date")))\
                        .withColumn("Year_Purchased", when(col("order_purchase_timestamp")!="null",year("order_purchase_timestamp")))\
                        .withColumn("Month_Purchased", when(col("order_purchase_timestamp")!="null",month("order_purchase_timestamp")))\
                        .withColumn("Day_Purchased", when(col("order_purchase_timestamp")!="null",dayofmonth("order_purchase_timestamp")))\
                        .withColumn("Year_Estimated", when(col("order_estimated_delivery_date")!="null",year("order_estimated_delivery_date")))\
                        .withColumn("Month_Estimated", when(col("order_estimated_delivery_date")!="null",month("order_estimated_delivery_date")))\
                        .withColumn("Day_Estimated", when(col("order_estimated_delivery_date")!="null",dayofmonth("order_estimated_delivery_date")))\
                        .withColumn("Delivered_Days",col("Year_delivered")*365+col("Month_delivered")*30+col("Day_delivered")
                                   -col("Year_Purchased")*365-col("Month_Purchased")*30-col("Day_Purchased"))\
                        .withColumn("Delivered_Est_Days", col("Year_Estimated")*365+col("Month_Estimated")*30+col("Day_Estimated")
                                   -col("Year_Purchased")*365-col("Month_Purchased")*30-col("Day_Purchased"))\
                        .withColumn("Difference", col("Delivered_Est_Days")-col("Delivered_Days"))

timeline_dates.show(10)

+--------------------+------------+-----------------------------+------------------------+-----------------------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+------------------+----------+
|         customer_id|order_status|order_delivered_customer_date|order_purchase_timestamp|order_estimated_delivery_date|Year_delivered|Month_delivered|Day_delivered|Year_Purchased|Month_Purchased|Day_Purchased|Year_Estimated|Month_Estimated|Day_Estimated|Delivered_Days|Delivered_Est_Days|Difference|
+--------------------+------------+-----------------------------+------------------------+-----------------------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+------------------+----------+
|9ef432eb625129730...|   delivered|          2017-10-10 21:25:13|     2017-10-02 10:56:33|       

In [85]:
timeline_dates_analysis = timeline_dates.groupBy("order_status")\
                                        .agg(avg("Delivered_Days").alias("avg_deliv_time"),
                                            avg("Delivered_Est_Days").alias("avg_est_del_time"))

for c in ['avg_deliv_time','avg_est_del_time']:
    timeline_dates_analysis = timeline_dates_analysis.withColumn(c, round(c, 2))

timeline_dates_analysis.show(10)

+------------+--------------+----------------+
|order_status|avg_deliv_time|avg_est_del_time|
+------------+--------------+----------------+
|     shipped|          null|           25.71|
|    canceled|         20.33|           23.19|
|    approved|          null|            26.0|
|    invoiced|          null|           25.88|
|     created|          null|            29.8|
|   delivered|          12.5|           24.35|
| unavailable|          null|           24.13|
|  processing|          null|           30.76|
+------------+--------------+----------------+



In [133]:
timeline_dates_analysis_2 = timeline_dates.groupBy("Year_delivered")\
                                        .agg(avg("Delivered_Days").alias("avg_deliv_time"),
                                            avg("Delivered_Est_Days").alias("avg_est_del_time"))

for c in ['avg_deliv_time','avg_est_del_time']:
    timeline_dates_analysis_2 = timeline_dates_analysis_2.withColumn(c, round(c, 2))

timeline_dates_analysis_2.show(10)

+--------------+--------------+----------------+
|Year_delivered|avg_deliv_time|avg_est_del_time|
+--------------+--------------+----------------+
|          2018|          12.8|           23.76|
|          null|          null|           25.34|
|          2016|         19.46|           55.08|
|          2017|         12.05|           24.95|
+--------------+--------------+----------------+



Reliability in terms of accuracy for a delivery service it's very important. The client wants to know exactly the arrival of the purchase. We shouldn't accept more than 10 days of difference between the estimation date arrival and the real one. According to this, we have categorized the Difference column into 4 categories to have the correct view of our data

In [139]:
customers_delivery = customers_df.join(timeline_dates, "customer_id")\
                                .select("customer_id", "customer_zip_code_prefix","customer_city","customer_state",
                                        "Difference", "Year_Purchased")\
                                .withColumn("Reliability", when(col("Difference")<=3, "High") 
                                                        .when((col("Difference")>3) & (col("Difference")<=10), "Medium")
                                                        .when(col("Difference")>10, "Low")
                                                        .otherwise("Unknown"))

customers_delivery.show(10)

+--------------------+------------------------+--------------------+--------------+----------+--------------+-----------+
|         customer_id|customer_zip_code_prefix|       customer_city|customer_state|Difference|Year_Purchased|Reliability|
+--------------------+------------------------+--------------------+--------------+----------+--------------+-----------+
|9ef432eb625129730...|                    3149|           sao paulo|            SP|         8|          2017|     Medium|
|b0830fb4747a6c6d2...|                   47813|           barreiras|            BA|         6|          2018|     Medium|
|41ce2a54c0b03bf34...|                   75265|          vianopolis|            GO|        17|          2018|        Low|
|f88197465ea7920ad...|                   59296|sao goncalo do am...|            RN|        13|          2017|        Low|
|8ab97904e6daea886...|                    9195|         santo andre|            SP|        10|          2018|     Medium|
|503740e9ca751ccdd...|  

When we group by the Reliability we can see how reliable is the E-commerce platform

In [150]:
reliability_delivery = customers_delivery.groupBy("Reliability")\
                                        .agg(count("customer_id").alias("Nº_Customers"))\
                                        .orderBy(col("Nº_Customers").desc())

reliability_delivery.show(10)

+-----------+------------+
|Reliability|Nº_Customers|
+-----------+------------+
|        Low|       56631|
|     Medium|       27171|
|       High|       12674|
|    Unknown|        2965|
+-----------+------------+



In [152]:
reliability_delivery = customers_delivery.groupBy("Reliability", "customer_state")\
                                        .agg(count("customer_id").alias("Nº_Customers"))\
                                        .orderBy(col("Nº_Customers").desc(),col("Reliability"),col("customer_state").desc())

reliability_delivery.show(10)

+-----------+--------------+------------+
|Reliability|customer_state|Nº_Customers|
+-----------+--------------+------------+
|        Low|            SP|       21237|
|     Medium|            SP|       14422|
|        Low|            MG|        7649|
|        Low|            RJ|        7412|
|       High|            SP|        4836|
|        Low|            RS|        3694|
|        Low|            PR|        3289|
|     Medium|            RJ|        2705|
|     Medium|            MG|        2681|
|       High|            RJ|        2236|
+-----------+--------------+------------+
only showing top 10 rows



In [144]:
reliability_delivery = customers_delivery.groupBy("Reliability", "Year_Purchased")\
                                        .agg(count("customer_id").alias("Nº_Customers"))\
                                        .orderBy(col("Nº_Customers").desc(),col("Reliability"))

reliability_delivery.show(10)

+-----------+--------------+------------+
|Reliability|Year_Purchased|Nº_Customers|
+-----------+--------------+------------+
|        Low|          2018|       28989|
|        Low|          2017|       27384|
|     Medium|          2018|       15573|
|     Medium|          2017|       11592|
|       High|          2018|        8216|
|       High|          2017|        4450|
|    Unknown|          2017|        1675|
|    Unknown|          2018|        1233|
|        Low|          2016|         258|
|    Unknown|          2016|          57|
+-----------+--------------+------------+
only showing top 10 rows



The conclusions we got from the results given are very important because the E-commerce platform should improve the reliability of the delivery estimation. Customers satisfaction is the main important objective and we shouldn't fail to estimate the product arrival date. 